In [1]:
import tqdm
import itertools

import datamol as dm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from mood.constants import DOWNSTREAM_RESULTS_DIR
from mood.dataset import MOOD_DATASETS
from mood.representations import MOOD_REPRESENTATIONS
from mood.baselines import SUPPORTED_BASELINES
from mood.utils import get_outlier_bounds
from mood.metrics import Metric
from mood.visualize import axes_grid_iterator, plot_performance_over_distance

/home/cas/local/conda/envs/mood_v2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SUBDIR = "20221213"

In [3]:
in_dir = dm.fs.join(DOWNSTREAM_RESULTS_DIR, "dataframes", "compare_performance", SUBDIR)
pattern = dm.fs.join(in_dir, "perf_over_distance_*.csv")
paths = dm.fs.glob(pattern)

In [4]:
def find_missing(paths, all_triplets):
    triplets = []
    for path in paths:
        path = dm.fs.get_basename(path)
        path = ".".join(path.split(".")[:-1])
        triplet = tuple(path.split("_")[3:])
        triplets.append(triplet)
    missing = set(all_triplets) - set(triplets)
    
    msg = f"Missing {len(missing)}:\n"
    for m in sorted(missing): 
        msg += f"{m}\n"
    return msg 
                        

all_triplets = list(itertools.product(MOOD_DATASETS, SUPPORTED_BASELINES, MOOD_REPRESENTATIONS))
assert list(all_triplets) == len(paths), find_missing(paths, all_triplets)

AssertionError: Missing 77:
('BBB', 'GP', 'ChemBERTa')
('BBB', 'GP', 'Desc2D')
('BBB', 'GP', 'ECFP6')
('BBB', 'GP', 'Graphormer')
('BBB', 'GP', 'MACCS')
('BBB', 'GP', 'WHIM')
('BBB', 'MLP', 'ChemBERTa')
('BBB', 'MLP', 'Desc2D')
('BBB', 'MLP', 'ECFP6')
('BBB', 'MLP', 'Graphormer')
('BBB', 'MLP', 'MACCS')
('BBB', 'MLP', 'WHIM')
('BBB', 'RF', 'ChemBERTa')
('BBB', 'RF', 'Desc2D')
('BBB', 'RF', 'ECFP6')
('BBB', 'RF', 'Graphormer')
('BBB', 'RF', 'MACCS')
('BBB', 'RF', 'WHIM')
('CYPP4502C9', 'GP', 'ChemBERTa')
('CYPP4502C9', 'GP', 'Desc2D')
('CYPP4502C9', 'GP', 'ECFP6')
('CYPP4502C9', 'GP', 'Graphormer')
('CYPP4502C9', 'GP', 'MACCS')
('CYPP4502C9', 'GP', 'WHIM')
('CYPP4502C9', 'MLP', 'ChemBERTa')
('CYPP4502C9', 'MLP', 'Desc2D')
('CYPP4502C9', 'MLP', 'ECFP6')
('CYPP4502C9', 'MLP', 'Graphormer')
('CYPP4502C9', 'MLP', 'MACCS')
('CYPP4502C9', 'MLP', 'WHIM')
('CYPP4502C9', 'RF', 'ChemBERTa')
('CYPP4502C9', 'RF', 'Desc2D')
('CYPP4502C9', 'RF', 'ECFP6')
('CYPP4502C9', 'RF', 'Graphormer')
('CYPP4502C

In [5]:
df_original = pd.concat([pd.read_csv(p) for p in tqdm.tqdm(paths)], ignore_index=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:27<00:00,  4.34it/s]


In [ ]:
for baseline, group in df_original.groupby("algorithm"):
    
    iterator = axes_grid_iterator(
        col_labels=MOOD_DATASETS, 
        row_labels=MOOD_REPRESENTATIONS
    )
    
    for ax, representation, dataset in iterator:
        df = group[group["dataset"] == dataset]
        df = df[df["representation"] == representation]
        
        performance_data = df[df["type"] == "performance"]
        calibration_data = df[df["type"] == "calibration"]
        
        plot_performance_over_distance(
            performance_data, 
            calibration_data, 
            dataset,
            ax=ax, 
            show_legend=False
        )

The End. 